In [1]:
import numpy as np
from numpy.linalg import svd as svd
from numpy.linalg import inv as inv

In [5]:
n = 50

def foo(i, j, k, l):
    return 1 / (i + j + k + l + 1)

A = np.fromfunction(foo, [n, n, n, n])
d = A.ndim

# Utils

In [7]:
def crop_vector(a: np.array, eps: float) -> int:
    """crop vector of positive values until all sum minus prefix sum < eps

    Args:
        a (np.array): vector of positive values
        eps (float): tolerance

    Returns:
        int: len of prefix sum
    """    
    r = 0
    cur_sum = 0
    sum = np.sum(a)

    while r < len(a):
        cur_sum += a[r]
        if sum - cur_sum < eps:
            return r + 1
        r += 1
    return r

def croped_svd(a: np.array, eps: float, full_matrices = False):
    """return cropped svd with absolute tolerance eps

    Args:
        a (np.array): matrix
        eps (float): absolute
        full_matrices (bool, optional): full_matrices for np.svd. Defaults to False.

    Returns:
        _type_: u, s, v - cropped svd
    """
    u, s, v = svd(a, full_matrices = full_matrices)
    r = crop_vector(s, eps)
    return u[:, :r], s[:r], v[:r, :]

# TT SVD

In [8]:
def TTSVD(A, eps): #from left to right
    d = A.ndim
    TT = A.reshape(A.shape[0], -1)
    u, s, v = croped_svd(TT, eps / (d**0.5))
    r1 = len(s)
    G1 = u
    TT = np.diag(s) @ v
    ranks = [r1]
    G = [G1]
    for k in range(1, d-1):
        TT = TT.reshape(ranks[k - 1] * A.shape[k], -1)
        u, s, v = svd(TT, full_matrices=False)
        ranks.append(crop_vector(s, eps / (d**0.5)))
        Gk = u[:, :ranks[k]].reshape(ranks[k-1], A.shape[k], ranks[k])
        G.append(Gk)
        TT = np.diag(s[:ranks[k]]) @ v[:ranks[k], :]
    G.append(TT)
    return G

In [9]:
G = TTSVD(A, 1e-8)
print([g.shape for g in G])

[(50, 13), (13, 50, 13), (13, 50, 13), (13, 50)]


In [268]:
def unfold_TT(G):
    d = len(G)
    g = G[0]
    for k in range(1, d-1):
        g = np.einsum(g, [i for i in range(g.ndim)], G[k], [g.ndim-1, g.ndim, g.ndim+1], [i for i in range(g.ndim - 1)] + [g.ndim, g.ndim+1]) # ...k,k...->...
    g = np.einsum(g, [i for i in range(g.ndim)], G[-1], [g.ndim - 1, g.ndim], [i for i in range(g.ndim -1)] + [g.ndim])
    return g

In [11]:
g = unfold_TT(G)
np.linalg.norm(g - A)

4.4758827043168295e-09

In [12]:
[np.linalg.norm(g)**2  for g in G], np.linalg.norm(g)**2

([12.999999999999996,
  12.999999999999986,
  12.999999999999986,
  952.4124701079983],
 952.4124701079974)

# Recompression

In [24]:
def recompess(G, eps, orthogonalization=False): #from right to left
    d = len(G)
    if orthogonalization:
        q, r = np.linalg.qr(G[0])
        G[0] = q
        np.einsum("ij,jkl->ikl", r, G[1])
        for k in range(1, d-2):
            q, r = np.linalg.qr(G[k].reshape(G[k-1].shape[-1] * G[k].shape[1], -1))
            G[k] = q.reshape(G[k-1].shape[-1], G[k].shape[1], G[k+1].shape[0])
            np.einsum("ij,jkl->ikl", r, G[k+1])
        G[-1] = r @ G[-1]

    u, s, v = croped_svd(G[d-1], eps / (d ** 0.5))
    Gnew = [v]
    W = u @ np.diag(s)
    for k in range(d-2, 0, -1):
        Gk = np.einsum("ijk,kl->ijl", G[k], W)
        u, s, v = croped_svd(Gk.reshape(Gk.shape[0], -1), eps / (d ** 0.5))
        W = u @ np.diag(s)
        Gnew.append(v.reshape(len(s), G[k].shape[1], Gnew[d-2-k].shape[0]))
    Gd = np.einsum("ij,jk->ik", G[0], W)
    u, s, v = croped_svd(Gd, eps / (d ** 0.5))
    Gnew.append(u @ np.diag(s) @ v)
    return Gnew[::-1]

In [25]:
Gnew = recompess(G, 1e-3, orthogonalization=True)
print("old shape:", [g.shape for g in G], "\n" "new shape:", [g.shape for g in Gnew])

old shape: [(50, 13), (13, 50, 13), (13, 50, 13), (13, 50)] 
new shape: [(50, 7), (7, 50, 8), (8, 50, 7), (7, 50)]


In [26]:
g1 = unfold_TT(G)
g2 = unfold_TT(Gnew)
np.linalg.norm(g1 - g2)

0.00033376470854269805

# TT cross


In [596]:
n = 5

def foo(i, j, k, l):
    return np.sin(i + j + k + l)

A = np.fromfunction(foo, [n, n, n, n])
d = A.ndim

In [597]:
def Sherman_Morrison_inv(A_inv, u, v):
    den = 1 + np.dot(v, A_inv @ u)
    return A_inv - np.outer(A_inv @ u,  v @ A_inv) / den

def MaxVol(A, eps=1e-2, maxit=20):
    m, r = A.shape


    for _try in range(20):
        set_i = np.random.choice(m, r, replace=False)
        cur_vol = abs(np.linalg.det(A[set_i, :]))
        if(cur_vol > 1e-5):
            break
    C = A[list(set_i), :]
    C_inv = np.linalg.inv(C)    
    for it in range(1, maxit):
        A_temp = np.abs(A @ C_inv)
        i, k = np.unravel_index(A_temp.argmax(), A_temp.shape)
        a_i = A[i, :]
        a_k = A[k, :]
        
        if A_temp[i, k] <= 1 + eps:
            break
        set_i[k] = i
        cur_vol *= A_temp[i, k]
        
        # C^-1 := (C - e_k(a_i - a_k))^-1
        C_inv = Sherman_Morrison_inv(C_inv, np.eye(1, r, k=k).ravel(), a_i - a_k)
    return np.sort(np.array(set_i))

def tensor_unfold(A: np.array, k : int) -> np.array:
    """unfold tensor by dimension k

    Args:
        A (np.aray): of size (n1, n2, ..., nd)
        k (int): dimension for unfold

    Returns:
        aray: matrix (nk, -1)
    """    
    return np.reshape(np.moveaxis(A, k, 0), (A.shape[k], -1))


In [1461]:
def TTcross(A_func, shape, ranks, eps = 1e-3, maxit=2, maxitcross = 100):
    d = len(shape)
    size = np.prod(shape)
    J = [np.random.choice(size // np.prod(shape[:k+1]), ranks[k], replace=False) for k in range(d-1)]
    I = [None for _ in range(d-1)]
    G = [None for _ in range(d)]
    for it in range(maxit):
        print(it)
        # forward iteration
        _slice = A_func(np.arange(shape[0]), *np.unravel_index(J[0], A.shape[1:]))
        I[0] = MaxVol(_slice, eps, maxitcross)
        for k in range(1, d-1):
            I[k] = np.array([i * A.shape[k-1] + j for i in I[k-1] for j in range(A.shape[k])])
            _slice = A_func(*np.unravel_index(I[k][:, None], shape[:k+1]), *np.unravel_index(J[k][None, :], shape[k+1:]))
            reduce_i = MaxVol(_slice, eps, maxitcross)
            I[k] = I[k][reduce_i]
        # backward iteration
        _slice = A_func(*np.unravel_index(I[-1], shape[:-1]), np.arange(A.shape[-1]))
        J[-1] = MaxVol(_slice.T, eps, maxitcross) # transpose for horizontal matrix
        for k in range(len(J) - 2, -1, -1):
            J[k] = np.array([j * shape[k+1] + i for j in J[k+1] for i in range(A.shape[k])])
            _slice = A(*np.unravel_index(I[k][:, None], shape[:k+1]), *np.unravel_index(J[k][None, :], shape[k+1:]))
            reduce_j = MaxVol(_slice.T, eps, maxitcross) # transpose for horizontal matrix
            J[k] = J[k][reduce_j]
    # creating tensor train
    _slice = A_func(np.arange(shape[0]), *np.unravel_index(J[0], shape[1:]))
    G[0] = _slice @ inv(_slice[I[0], :])
    for k in range(1, d-1):
        I[k] = np.array([i * A.shape[k-1] + j for i in I[k-1] for j in range(A.shape[k])])
        _slice = A_func(*np.unravel_index(I[k][:, None], shape[:k+1]), *np.unravel_index(J[k][None, :], shape[k+1:]))
        reduce_i = MaxVol(_slice, eps, maxitcross)
        g = _slice @ inv(_slice[reduce_i, :])
        I[k] = I[k][reduce_i]
        G[k] = g.reshape(ranks[k-1], A.shape[k], ranks[k])
    G[-1] = A_func(*np.unravel_index(I[-1], A.shape[:-1]), np.arange(A.shape[-1]))

    return G

In [1478]:
ranks = [2 for _ in range(A.ndim)]
G = TTcross(foo, A.shape, ranks, maxit=3)

0
1
2


In [1479]:
g = unfold_TT(G)
np.linalg.norm(g - A)

1.0711135194699271e-14